In [9]:
import numpy as np
import netCDF4 as nc
import auxiliary as au
import regression as reg
import datetime as dt
from matplotlib import pyplot as plt
from scipy import io
import os
import sys

########################################################################################################################
# date_cal_start = int(sys.argv[1]) # yyyymmdd
# date_cal_end = int(sys.argv[2])

# 0. read/define configuration information
# setting: file and path names of inputs
FileStnInfo = '/Users/localuser/GMET/pyGMET_NA/stnlist_whole.txt'  # station basic information (lists)
FileGridInfo = '/Users/localuser/GMET/pyGMET_NA/gridinfo_whole.nc'  # study area information
PathStn = '/Users/localuser/GMET/StnInput_daily'
# Plato
# FileStnInfo = '/home/gut428/GMET/eCAI_EMDNA/StnGridInfo/stnlist_whole.txt'  # station basic information (lists)
# FileGridInfo = '/home/gut428/GMET/eCAI_EMDNA/StnGridInfo/gridinfo_whole.nc'  # study area information
# PathStn = '/home/gut428/GMET/StnInput_daily'

# setting: start and end date
# calculation start/end date:
date_cal_start = 19790501  # yyyymmdd: start date
date_cal_end = 19790531  # yyyymmdd: end date
# station data (in PathStn) start/end date:
date_stn_start = 19790101  # yyyymmdd: start date
date_stn_end = 20181231  # yyyymmdd: end date

# setting: paramters for lag correlation of tmean_stn_daily, and cross-correlation between prcp and trange_stn_daily
windows = 1  # parameters for auto-cc t-p-cc calculation: 1 could be better than 31
lag = 1

# setting: searching nearby stations
nearstn_min = 20  # nearby stations: minimum number
nearstn_max = 30  # nearby stations: maximum number
search_radius = 400  # km. only search stations within this radius even nearstn_max cannot be reached
max_dist = 100  # max_distance in distance-based weight calculation

# setting: parameters for transforming temp to approximate normal distribution
trans_mode = 'none'  # box-cox or power-law or none
trans_exp_daily = 4

# setting: overwrite flags. -1:don't save files; 0: don't overwrite files; 1 is to overwrite existing files;
ow_daily = 0
ow_weight = 0
ow_stn = 0

# setting: output files
datestr = str(date_cal_start) + '-' + str(date_cal_end)
# FileStnData = '/home/gut428/GMET/PyGMETout/stndata_' + datestr + '.npz'
# FileWeight = '/home/gut428/GMET/PyGMETout/weight.npz'
# FileRegError_daily = '/home/gut428/GMET/PyGMETout/error_' + datestr + '.npz'  # regression error at station points
# FileRegError_daily_corr = '/home/gut428/GMET/PyGMETout/error_rescorr' + datestr + '.npz'  # regression error after residual correction
# FileRegression_daily = '/home/gut428/GMET/PyGMETout/output_' + datestr + '.npz'
FileStnData = '/Users/localuser/GMET/error_compare/stndata_' + datestr + '.npz'
FileWeight = '/Users/localuser/Downloads/old/weight.npz'
FileRegError_daily = '/Users/localuser/GMET/error_compare/error_notrans_' + datestr + '.npz'  # regression error at station points
FileRegError_daily_corr = '//Users/localuser/Downloads/error_rescorr' + datestr + '.npz'
FileRegression_daily = '/Users/localuser/Downloads/output_' + datestr + '.npz'

########################################################################################################################

# 1. basic information

print('Read study area basic information')
# station location and attribute information
# stninfo: [ stations, 1/lat/lon/elev/slope_ns/slope_we ]
stnID, stninfo = au.readstnlist(FileStnInfo)
nstn = len(stnID)

# time information
if date_cal_start < date_stn_start:
    sys.exit('The calculation period is earlier than the station period')
if date_cal_end > date_stn_end:
    sys.exit('The calculation period is later than the station period')

date_cal_start2 = dt.datetime.strptime(str(date_cal_start), '%Y%m%d')
date_cal_end2 = dt.datetime.strptime(str(date_cal_end), '%Y%m%d')
ntimes = (date_cal_end2 - date_cal_start2).days + 1  # time steps to be processed

date_stn_start2 = dt.datetime.strptime(str(date_stn_start), '%Y%m%d')
loc_start = (date_cal_start2 - date_stn_start2).days  # start location in the netcdf file
loc_end = loc_start + ntimes

# seconds since 1970-1-1 0:0:0
daydiff = (date_cal_start2 - dt.datetime(1970, 1, 1)).days
seconds = (np.arange(ntimes) + daydiff) * 86400

# datelist: yyyymmdd
yyyymmdd = np.zeros(ntimes, dtype=int)
for d in range(ntimes):
    dated = date_cal_start2 + dt.timedelta(days=d)
    yyyymmdd[d] = int(dated.strftime("%Y%m%d"))
yyyymm = np.floor(yyyymmdd / 100).astype(int)
mm = np.floor(np.mod(yyyymmdd, 10000) / 100).astype(int)

########################################################################################################################

# 2. read study area basic information
print('Read study area basic information')
ncfid = nc.Dataset(FileGridInfo)
gridlat = ncfid.variables['latitude'][:].data
gridlon = ncfid.variables['longitude'][:].data
gridele = ncfid.variables['elev'][:].data
gridgns = ncfid.variables['gradient_n_s'][:].data
gridgwe = ncfid.variables['gradient_w_e'][:].data
mask = ncfid.variables['mask'][:].data  # 1: grids to be considered; the other values: invalid grids
ncfid.close()

nrows, ncols = np.shape(gridlat)
gridinfo = np.zeros([nrows, ncols, 6])
gridinfo[:, :, 0] = 1
gridinfo[:, :, 1] = gridlat
gridinfo[:, :, 2] = gridlon
gridinfo[:, :, 3] = gridele
gridinfo[:, :, 4] = gridgns
gridinfo[:, :, 5] = gridgwe
del gridlat, gridlon, gridele, gridgns, gridgwe

########################################################################################################################

# 3. read data (prcp, tmin, tmax) from station files
print('Read station precipitation and temperature data')
if os.path.isfile(FileStnData) and ow_stn != 1:
    print('FileStnData exists. loading ...')
    with np.load(FileStnData) as datatemp:
        prcp_stn_daily = datatemp['prcp_stn_daily']
        tmean_stn_daily = datatemp['tmean_stn_daily']
        trange_stn_daily = datatemp['trange_stn_daily']
        prcp_stn_climo = datatemp['prcp_stn_climo']
        tmean_stn_climo = datatemp['tmean_stn_climo']
        trange_stn_climo = datatemp['trange_stn_climo']
        prcp_stn_anom = datatemp['prcp_stn_anom']
        tmean_stn_anom = datatemp['tmean_stn_anom']
        trange_stn_anom = datatemp['trange_stn_anom']
else:
    cai_mode = 0
    prcp_stn_daily, tmean_stn_daily, trange_stn_daily, \
    prcp_stn_climo, tmean_stn_climo, trange_stn_climo, \
    prcp_stn_anom, tmean_stn_anom, trange_stn_anom \
        = au.read_station(PathStn, stnID, loc_start, loc_end, cai_mode, yyyymm)
    np.savez_compressed(FileStnData,
                        prcp_stn_daily=prcp_stn_daily, tmean_stn_daily=tmean_stn_daily, trange_stn_daily=trange_stn_daily)
    del prcp_stn_climo, tmean_stn_climo, trange_stn_climo, prcp_stn_anom, tmean_stn_anom, trange_stn_anom

########################################################################################################################

# 4. calculate auto_corr and t_p_corr
print('Calculate correlation (auto_cc and t_p_cc)')
mean_autocorr_daily, mean_tp_corr_daily = au.cc_calculate(windows, lag, prcp_stn_daily, tmean_stn_daily,
                                                          trange_stn_daily)
print('Tmean lag-1 daily autocorrelation: ', mean_autocorr_daily)
print('Trange-prcp daily correlation: ', mean_tp_corr_daily)

########################################################################################################################

# 5. find neighboring stations and calculate distance-based weights
if os.path.isfile(FileWeight) and ow_weight != 1:
    print('FileWeight exists. loading ...')
    with np.load(FileWeight) as datatemp:
        near_grid_prcpLoc = datatemp['near_grid_prcpLoc']
        near_grid_prcpWeight = datatemp['near_grid_prcpWeight']
        near_grid_tempLoc = datatemp['near_grid_tempLoc']
        near_grid_tempWeight = datatemp['near_grid_tempWeight']
        near_stn_prcpLoc = datatemp['near_stn_prcpLoc']
        near_stn_prcpWeight = datatemp['near_stn_prcpWeight']
        near_stn_tempLoc = datatemp['near_stn_tempLoc']
        near_stn_tempWeight = datatemp['near_stn_tempWeight']
    del datatemp
else:
    near_grid_prcpLoc, near_grid_prcpDist, near_grid_prcpWeight, \
    near_grid_tempLoc, near_grid_tempDist, near_grid_tempWeight, \
    near_stn_prcpLoc, near_stn_prcpDist, near_stn_prcpWeight, \
    near_stn_tempLoc, near_stn_tempDist, near_stn_tempWeight \
        = au.station_weight(prcp_stn_daily, tmean_stn_daily, stninfo, gridinfo, mask,
                            search_radius, nearstn_min, nearstn_max, max_dist)

    # save data
    np.savez_compressed(FileWeight, near_grid_prcpLoc=near_grid_prcpLoc, near_grid_prcpDist=near_grid_prcpDist,
                        near_grid_prcpWeight=near_grid_prcpWeight, near_grid_tempLoc=near_grid_tempLoc,
                        near_grid_tempDist=near_grid_tempDist, near_grid_tempWeight=near_grid_tempWeight,
                        near_stn_prcpLoc=near_stn_prcpLoc, near_stn_prcpDist=near_stn_prcpDist,
                        near_stn_prcpWeight=near_stn_prcpWeight, near_stn_tempLoc=near_stn_tempLoc,
                        near_stn_tempDist=near_stn_tempDist, near_stn_tempWeight=near_stn_tempWeight)

########################################################################################################################

# 6. start spatial regression

########################################################################################################################

# 6.1 estimate regression error at station points
if os.path.isfile(FileRegError_daily) and ow_daily != 1:
    print('FileRegError_daily exists. loading ...')
    with np.load(FileRegError_daily) as datatemp:
        pcp_err_stn_daily = datatemp['pcp_err_stn']
#         tmean_err_stn_daily = datatemp['tmean_err_stn']
#         trange_err_stn_daily = datatemp['trange_err_stn']
    del datatemp
else:
    print('Estimate daily regression error at station points')
    pcp_err_stn_daily, tmean_err_stn_daily, trange_err_stn_daily = \
        reg.station_error(prcp_stn_daily, tmean_stn_daily, trange_stn_daily, stninfo, near_stn_prcpLoc,
                          near_stn_prcpWeight, near_stn_tempLoc, near_stn_tempWeight, trans_exp_daily,
                          trans_mode, nearstn_min)
    np.savez_compressed(FileRegError_daily, pcp_err_stn=pcp_err_stn_daily, tmean_err_stn=tmean_err_stn_daily,
                        trange_err_stn=trange_err_stn_daily, stninfo=stninfo)

Read study area basic information
Read study area basic information
Read station precipitation and temperature data
FileStnData exists. loading ...
Calculate correlation (auto_cc and t_p_cc)
Tmean lag-1 daily autocorrelation:  0.6574480697293462
Trange-prcp daily correlation:  -0.29674016420547195
FileWeight exists. loading ...
FileRegError_daily exists. loading ...


In [10]:
pcp_reg_ori=prcp_stn_daily+pcp_err_stn_daily
metori=np.zeros([nstn,4])
for i in range(nstn):
    metori[i,:] = au.metric(prcp_stn_daily[i,:],pcp_reg_ori[i,:])
print('original metric is')
print(np.nanmean(metori,axis=0))

original metric is
[0.76894048 0.02818352 1.69320388 3.61459002]


In [11]:
pcp_err_stn_daily_corr = reg.error_after_residualcorrection(prcp_stn_daily, pcp_reg_ori, near_stn_prcpLoc, near_stn_prcpWeight)
pcp_reg_corr=prcp_stn_daily+pcp_err_stn_daily_corr
metcorr=np.zeros([nstn,4])
for i in range(nstn):
    metcorr[i,:] = au.metric(prcp_stn_daily[i,:],pcp_reg_corr[i,:])
print('corrected metric is')
print(np.nanmean(metcorr,axis=0))

corrected metric is
[7.43183331e-01 3.32785081e-03 1.76720025e+00 3.77749940e+00]


In [49]:
file='/Users/localuser/Research/Test/mergecorr_trange_RMSE.npz'
datatemp = np.load(file)
reamerge_stn = datatemp['reamerge_stn']
reamerge_weight_stn = datatemp['reamerge_weight_stn']
reacorr_stn = datatemp['reacorr_stn']

In [47]:
gmet_stndatafile = '/Users/localuser/Downloads/stndata_whole.npz'
datatemp = np.load(gmet_stndatafile)
prcp_stn = datatemp['trange_stn']

In [48]:
metmerge=np.zeros([nstn,4])
metcorr=np.zeros([3,nstn,4])
for i in range(nstn):
    metmerge[i,:]=au.metric(prcp_stn[i,:],reamerge_stn[i,:])
    for j in range(3):
        metcorr[j,i,:]=au.metric(prcp_stn[i,:],reacorr_stn[j, i,:])
print(np.nanmean(metmerge,axis=0))
for j in range(3):
    print(np.nanmean(metcorr[j,:,:],axis=0))

/usr/local/lib/python3.7/site-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/localuser/Github/PyGMET/auxiliary.py:150: RuntimeWarning: Mean of empty slice
  metout[1] = np.nanmean(pre - obs)  # ME
/Users/localuser/Github/PyGMET/auxiliary.py:151: RuntimeWarning: Mean of empty slice
  metout[2] = np

[0.75699575 0.01854413 2.45493013 3.16221866]
[0.75558377 0.00402423 2.45591684 3.16678333]
[0.75158845 0.021099   2.49609959 3.21397457]
[0.74154005 0.03895172 2.56515831 3.29929942]


In [50]:
metmerge=np.zeros([nstn,4])
metcorr=np.zeros([3,nstn,4])
for i in range(nstn):
    metmerge[i,:]=au.metric(prcp_stn[i,:],reamerge_stn[i,:])
    for j in range(3):
        metcorr[j,i,:]=au.metric(prcp_stn[i,:],reacorr_stn[j, i,:])
print(np.nanmean(metmerge,axis=0))
for j in range(3):
    print(np.nanmean(metcorr[j,:,:],axis=0))

[0.75733972 0.01808528 2.45381974 3.16063678]
[0.75558377 0.00402423 2.45591684 3.16678333]
[0.75158845 0.021099   2.49609959 3.21397457]
[0.74154005 0.03895172 2.56515831 3.29929942]


In [33]:
metcorr[:,10,1]

array([-2.31306815, -1.94949377, -2.2570343 ])

In [43]:
datatemp['trange_stn']

BadZipFile: Bad CRC-32 for file 'trange_stn.npy'

In [60]:
file='/Users/localuser/Downloads/output_notrans_19790101-19790131.npz'
d=np.load(file)
dm=d['pcp']

In [75]:
a=np.reshape(np.arange(30),[2,3,5])
a[0,1,:]

array([5, 6, 7, 8, 9])

In [73]:
np.reshape(a,[6,5])

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29]])

In [87]:
a = dict()
a['t1'] = np.arange(3)
a['t2'] = np.arange(4)